In [20]:
import io
import os
import math
import base64
import rdkit
import ipywidgets as widgets
import numpy as np
import pandas as pd
import IPython

In [2]:
def rdimage(mol: rdkit.Chem.Mol) -> bytes:
    """Returns an image of a molecule as bytes."""
    pil_image = rdkit.Chem.Draw.MolToImage(mol)
    buf = io.BytesIO()
    pil_image.save(buf, format='png')
    return buf.getvalue()

In [3]:
# filename = "drd2-subset300mols.csv"
# Can also use FileUpload widget: 
# https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html#File-Upload
# df = pd.read_csv(filename)

In [4]:
uploader = widgets.FileUpload(
    accept='.csv,csv.gz',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False  # True to accept multiple files upload else False
)
display(uploader)

FileUpload(value={}, accept='.csv,csv.gz', description='Upload')

In [6]:
uploaded_file = next(iter(uploader.value.values()))  # ipywidgets 7.6
# uploaded_file = uploader.value[0]  # ipywidgets 8.0

In [7]:
df = pd.read_csv(io.BytesIO(uploaded_file["content"]))

In [8]:
df["mol"] = df["canonical"].apply(rdkit.Chem.MolFromSmiles)

In [9]:
df["mol_widget"] = df["mol"].apply(
    lambda mol: widgets.Image(
        value=rdimage(mol),
        format="PNG",
        width=150,
        height=150,
    )
)

In [10]:
df["button_widget"] = df["canonical"].apply(
    lambda smi: widgets.ToggleButton(
        value=False, 
        description=smi,
        layout=widgets.Layout(height="auto", width="auto")
    )
)

In [11]:
df["duo_widget"] = df.apply(
    axis=1, 
    func=lambda row: widgets.VBox(
        [row.mol_widget, row.button_widget],
        layout=widgets.Layout(margin="5px 5px 40px 5px")  # top, right, bottom, left
    )
)

In [12]:
N_COLS = 4
N_ROWS = math.ceil(len(df) / N_COLS)
grid = widgets.GridspecLayout(N_ROWS, N_COLS)

In [13]:
for i in range(len(df)):
    (row, col) = np.unravel_index(i, (N_ROWS, N_COLS))
    grid[row, col] = df["duo_widget"][i]

In [14]:
grid

GridspecLayout(children=(VBox(children=(Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01,\x00\x00…

In [15]:
df["is_selected"] = df["button_widget"].map(lambda row: row.value)

auxiliary_ui_columns = ["mol", "mol_widget", "button_widget", "duo_widget"]
good_columns = df.columns.difference(auxiliary_ui_columns)
df2 = df[good_columns]

df2

,activity,canonical,is_selected,molwt,molwt_gt_330
0,0,CCCC=N[SH](=O)(O)C=Cc1ccccc1,False,239.340,False
1,0,O=C(Nc1ccc2c(c1)OCO2)c1cnn2cccnc12,False,282.259,False
2,0,N#Cc1c(N)n(CCNC(=O)c2cccs2)c2nc3ccccc3nc12,False,362.418,True
3,0,CC(=O)c1cccc(NC(=O)Cn2cnc3c2c(=O)n(C)c(=O)n3C)c1,False,355.354,True
4,0,CCOC(=O)c1oc2ccccc2c1NC(=O)Cc1cccs1,False,329.377,False
...,...,...,...,...,...
295,0,O=C(O)c1cccc([N+](=O)[O-])c1C(=O)Nc1nc2ccccc2s1,False,343.320,True
296,0,CN(C)S(=O)(=O)c1cccc(NC(=O)COC(=O)COc2ccc(Br)c...,False,471.329,True
297,0,CCOc1ccc(NC(=O)CCC(=O)c2ccc(Br)cc2)cc1,True,376.250,True
298,0,CC1CCN(S(=O)(=O)c2ccc3c(c2)c(=O)c(C(=O)NCc2ccc...,False,459.593,True


In [21]:
# https://stackoverflow.com/a/42907645
def create_download_link( df, title = "Download CSV file", filename = "data.csv"):
    csv = df.to_csv()
    payload = base64.b64encode(csv.encode()).decode()
    html = f'<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    return IPython.display.HTML(html)

create_download_link(df2)

In [22]:
# https://stackoverflow.com/a/62641240
def create_download_button( df, title = "Download CSV file", filename = "data.csv"):
    csv = df.to_csv()
    payload = base64.b64encode(csv.encode()).decode()

    html_button = f'''<html>
    <head>
    <meta name="viewport" content="width=device-width, initial-scale=1">
    </head>
    <body>
    <a download="{filename}" href="data:text/csv;base64,{payload}" download>
    <button class="p-Widget jupyter-widgets jupyter-button widget-button mod-warning">Download File</button>
    </a>
    </body>
    </html>
    '''
    # IPython.display
    display(widgets.HTML(html_button))

create_download_button(df2)

HTML(value='<html>\n    <head>\n    <meta name="viewport" content="width=device-width, initial-scale=1">\n    …

In [160]:
def out_filename(in_filename):
    name, ext = os.path.splitext(in_filename)
    return f"{name}-with-user-selection{ext}"

df2.to_csv(out_filename(filename), index=False)

In [23]:
df["mol_widget"][0].

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01,\x00\x00\x01,\x08\x02\x00\x00\x00\xf6\x1f\x19"\x…